Copyright (c) 2020, NVIDIA CORPORATION.

Modifications copyright Intel. 

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

###  BertTokenizer to analysis text_token, extract tweets 


In [1]:
import matplotlib.pyplot as plt
%matplotlib inline  

import pandas as pd
import numpy as np
import gc
from tqdm.notebook import tqdm

from transformers import *
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

import time 
from datetime import datetime 

In [2]:
time_begin = datetime.now()

In [3]:
%%time
df = pd.read_csv('/mnt/DP_disk3/Recsys/training.tsv', sep='\x01', header=None, usecols=[0] )
df.columns = ['text_tokens']
df['id']   = np.arange( df.shape[0] )
df['id']   = df['id'].astype(np.uint32)
gc.collect()
df.head()

CPU times: user 6min 15s, sys: 1min 21s, total: 7min 36s
Wall time: 7min 39s


,text_tokens,id
0,101\t60104\t112\t187\t29522\t12174\t10347\t348...,0
1,101\t16202\t15853\t23886\t10291\t11670\t88237\...,1
2,101\t62338\t17502\t94028\t10126\t100448\t10954...,2
3,101\t56898\t137\t91729\t50784\t77966\t11373\t1...,3
4,101\t56898\t137\t32718\t11447\t19584\t10112\t1...,4


In [4]:
%%time
dv = pd.read_csv('/mnt/DP_disk3/Recsys/val.tsv', sep='\x01', header=None, usecols=[0] )
dv.columns = ['text_tokens']
dv['id']   = np.arange( df.shape[0] , df.shape[0]+dv.shape[0] )
dv['id']   = dv['id'].astype(np.uint32)
gc.collect()
dv.head()

CPU times: user 33.5 s, sys: 6.5 s, total: 40 s
Wall time: 40.3 s


,text_tokens,id
0,101\t10117\t140\t119\t142\t119\t152\t119\t1010...,96600391
1,101\t10105\t10817\t10124\t59232\t18121\t15629\...,96600392
2,101\t48561\t10116\t67737\t18554\t36371\t10989\...,96600393
3,101\t100055\t69940\t10414\t159\t11305\t11166\t...,96600394
4,101\t62154\t32221\t71843\t10143\t10237\t15507\...,96600395


In [5]:
%%time
dt = pd.read_csv('/mnt/DP_disk3/Recsys/test.tsv', sep='\x01', header=None, usecols=[0] )
dt.columns = ['text_tokens']
dt['id']   = np.arange( df.shape[0]+dv.shape[0], df.shape[0]+dv.shape[0]+dt.shape[0] )
dt['id']   = dt['id'].astype(np.uint32)
gc.collect()
dt.head()

CPU times: user 33.9 s, sys: 7.21 s, total: 41.1 s
Wall time: 41.4 s


,text_tokens,id
0,101\t3100\t5477\t3028\t4348\t1924\t111806\t186...,106543692
1,101\t56898\t137\t36110\t10400\t168\t64062\t131...,106543693
2,101\t10117\t23672\t12610\t86532\t11205\t90138\...,106543694
3,101\t13690\t14372\t119\t119\t19281\t25444\t984...,106543695
4,101\t56898\t137\t171\t64791\t168\t11499\t10330...,106543696


In [6]:
%%time
df['tweet'] = [ tokenizer.decode( [ int(n) for n in t.split('\t') ] ) for t in tqdm(df.text_tokens.values) ] 
gc.collect()


CPU times: user 1h 32min 12s, sys: 3min 49s, total: 1h 36min 2s
Wall time: 1h 35min 11s


23

In [7]:
%%time
dv['tweet'] = [ tokenizer.decode( [ int(n) for n in t.split('\t') ] ) for t in tqdm(dv.text_tokens.values) ] 
gc.collect()


CPU times: user 9min 19s, sys: 21.7 s, total: 9min 41s
Wall time: 9min 37s


23

In [8]:
%%time
dt['tweet'] = [ tokenizer.decode( [ int(n) for n in t.split('\t') ] ) for t in tqdm(dt.text_tokens.values) ] 
gc.collect()


CPU times: user 9min 19s, sys: 21.6 s, total: 9min 40s
Wall time: 9min 35s


23

In [9]:
%%time

df['tweet'] = df['tweet'].apply( lambda x: x.replace('https : / / t. co / ', 'https://t.co/') )
df['tweet'] = df['tweet'].apply( lambda x: x.replace('@ ', '@') )
df.head()

CPU times: user 4min 23s, sys: 1min 33s, total: 5min 57s
Wall time: 5min 57s


,text_tokens,id,tweet
0,101\t60104\t112\t187\t29522\t12174\t10347\t348...,0,[CLS] Mirage's Ultimate could be getting chang...
1,101\t16202\t15853\t23886\t10291\t11670\t88237\...,1,[CLS] Dat zou thuis niet misstaan @deBijenkorf...
2,101\t62338\t17502\t94028\t10126\t100448\t10954...,2,"[CLS] Belajar naik sepeda aja kita terjatuh, t..."
3,101\t56898\t137\t91729\t50784\t77966\t11373\t1...,3,[CLS] RT @iamchie019 : Dara.. Dara.. Beautiful...
4,101\t56898\t137\t32718\t11447\t19584\t10112\t1...,4,[CLS] RT @JoyGwene : [UNK]. @ion @jugsjugsjugs...


In [10]:
%%time
dv['tweet'] = dv['tweet'].apply( lambda x: x.replace('https : / / t. co / ', 'https://t.co/') )
dv['tweet'] = dv['tweet'].apply( lambda x: x.replace('@ ', '@') )
dv.head()

CPU times: user 30.2 s, sys: 1.89 s, total: 32.1 s
Wall time: 32.1 s


,text_tokens,id,tweet
0,101\t10117\t140\t119\t142\t119\t152\t119\t1010...,96600391,"[CLS] The C. E. O. of Acronym, the tech consul..."
1,101\t10105\t10817\t10124\t59232\t18121\t15629\...,96600392,[CLS] the man is literally barking.. [SEP]
2,101\t48561\t10116\t67737\t18554\t36371\t10989\...,96600393,"[CLS] Usai juara dunia enam tahun lalu, timnas..."
3,101\t100055\t69940\t10414\t159\t11305\t11166\t...,96600394,[CLS] Här hör ni V75 - podden med Niclas och E...
4,101\t62154\t32221\t71843\t10143\t10237\t15507\...,96600395,"[CLS] Bursa trfr dah ditutup, lo ngapain @juve..."


In [11]:
%%time
dt['tweet'] = dt['tweet'].apply( lambda x: x.replace('https : / / t. co / ', 'https://t.co/') )
dt['tweet'] = dt['tweet'].apply( lambda x: x.replace('@ ', '@') )

dt.head()

CPU times: user 31.6 s, sys: 5.17 s, total: 36.8 s
Wall time: 36.8 s


,text_tokens,id,tweet
0,101\t3100\t5477\t3028\t4348\t1924\t111806\t186...,106543692,[CLS] 埼 玉 土 日 じゃん ！. 行 けないこともないか... ？ [UNK]. え...
1,101\t56898\t137\t36110\t10400\t168\t64062\t131...,106543693,[CLS] RT @meanie _ ark : いいね ・ RTでそれぞれポイントになるみ...
2,101\t10117\t23672\t12610\t86532\t11205\t90138\...,106543694,[CLS] The Evolution Of Adidas Predators ( via ...
3,101\t13690\t14372\t119\t119\t19281\t25444\t984...,106543695,[CLS] Special edition.. Democratic Kampuchea. ...
4,101\t56898\t137\t171\t64791\t168\t11499\t10330...,106543696,[CLS] RT @click _ burgundy : ＃ 一 日 一 描 ＃ ミンサガ ...


In [12]:
df.to_parquet( '/mnt/DP_disk3/Recsys/train-tweet-1.parquet' )
dv.to_parquet( '/mnt/DP_disk3/Recsys/test0-tweet-1.parquet' )
dt.to_parquet( '/mnt/DP_disk3/Recsys/test1-tweet-1.parquet' )

In [13]:
time_end = datetime.now()

print("3rd notebook total process time", time_end - time_begin)

3rd notebook total process time 2:19:02.096827


In [14]:
dv

,text_tokens,id,tweet
0,101\t10117\t140\t119\t142\t119\t152\t119\t1010...,96600391,"[CLS] The C. E. O. of Acronym, the tech consul..."
1,101\t10105\t10817\t10124\t59232\t18121\t15629\...,96600392,[CLS] the man is literally barking.. [SEP]
2,101\t48561\t10116\t67737\t18554\t36371\t10989\...,96600393,"[CLS] Usai juara dunia enam tahun lalu, timnas..."
3,101\t100055\t69940\t10414\t159\t11305\t11166\t...,96600394,[CLS] Här hör ni V75 - podden med Niclas och E...
4,101\t62154\t32221\t71843\t10143\t10237\t15507\...,96600395,"[CLS] Bursa trfr dah ditutup, lo ngapain @juve..."
...,...,...,...
9943296,101\t56898\t137\t44531\t11517\t12730\t10465\t1...,106543687,[CLS] RT @AuschwitzMuseum : 31 January 1931 | ...
9943297,101\t56898\t137\t10911\t17750\t37263\t10161\t1...,106543688,[CLS] RT @meyerlegal : Franca Werhahn @meyerle...
9943298,101\t56898\t137\t12187\t10245\t11517\t10157\t1...,106543689,[CLS] RT @99inMyMind : 200205 HBD TO [UNK]. [U...
9943299,101\t100\t12976\t100\t187\t10105\t10992\t12452...,106543690,[CLS] [UNK] what [UNK] s the most important th...
